In [1]:
# Some boilerplate setup
import sys
sys.path.append("../")

import time
import pickle
import numpy as np
import tensorflow as tf
from tensorflow.contrib.learn import infer_real_valued_columns_from_input
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

EPS = 1e-8 # epsilon constant for numeric stability

In [2]:
# Available agents/environments:
#  Ant-v1.pkl
#  HalfCheetah-v1.pkl
#  Hopper-v1.pkl
#  Humanoid-v1.pkl
#  Reacher-v1.pkl
#  Walker2d-v1.pkl

ENV_NAME = "Ant-v1"

## Expert data loading and preprocessing

In [4]:
# Load the expert data from pickle saved in 'homework1/expert_data'
expert_data = None
with open("../expert_data/{}.pkl".format(ENV_NAME), "rb") as f:
    expert_data = pickle.load(f)

observations = expert_data["observations"].astype('float32')
actions = np.squeeze(expert_data["actions"].astype('float32'))

# As a sanity check, print out the size of the training and test data.
print('observations shape: ', observations.shape)
print('actions shape: ', actions.shape)

num_total, D_in = observations.shape
D_out = actions.shape[-1]

observations shape:  (20000, 111)
actions shape:  (20000, 8)


## Train, validation, test split

In [10]:
# Split the data into train, val, and test sets. Also, create a
# small dataset for development purposes.
train_prop = 16/20
val_prop = 3/20
test_prop = 1/20

num_train = int(np.floor(num_total * train_prop))
num_val   = int(np.ceil(num_total * val_prop))
num_test  = num_total - num_train - num_val
num_dev   = 500

# Split the data into test set and temporary set, which will be 
# split into training and validation sets
X_tmp, X_test, y_tmp, y_test = train_test_split(observations,
                                                actions,
                                                test_size=num_test)

# Split X_tmp into training and validation
X_train, X_val, y_train, y_val = train_test_split(X_tmp,
                                                  y_tmp,
                                                  train_size=num_train)
# Create the development set, which is just a small subset of
# the training set.
mask = np.random.choice(num_train, num_dev, replace=False)
X_dev = X_train[mask]
y_dev = y_train[mask]

print('Train data: ', X_train.shape, X_train.dtype)
assert(X_train.shape[0] == num_train)
print('Train labels: ', y_train.shape, y_train.dtype)
assert(y_train.shape[0] == num_train)

print('Validation data: ', X_val.shape, X_val.dtype)
assert(X_val.shape[0] == num_val)
print('Validation labels: ', y_val.shape, y_val.dtype)
assert(y_val.shape[0] == num_val)

print('Test data: ', X_test.shape, X_test.dtype)
assert(X_test.shape[0] == num_test)
print('Test labels: ', y_test.shape, y_test.dtype)
assert(y_test.shape[0] == num_test)

print('Dev data: ', X_dev.shape, X_dev.dtype)
assert(X_dev.shape[0] == num_dev)
print('Dev labels: ', y_dev.shape, y_dev.dtype)
assert(y_dev.shape[0] == num_dev)

Train data:  (16000, 111) float32
Train labels:  (16000, 8) float32
Validation data:  (3000, 111) float32
Validation labels:  (3000, 8) float32
Test data:  (1000, 111) float32
Test labels:  (1000, 8) float32
Dev data:  (500, 111) float32
Dev labels:  (500, 8) float32


### Preprocessing: zero mean and unit variance

In [11]:
# Preprocessing: subtract the mean image
# first: compute the mean and standard deviation from training X
mean = np.nanmean(X_train, axis=0)
std = np.std(X_train, axis=0) + EPS

# second: subtract mean and std from datasets
for X in (X_train, X_val, X_test, X_dev):
    X -= mean
    X /= std

### Initialize things used by neural network models

In [12]:
feature_columns = infer_real_valued_columns_from_input(X_train)

validation_metrics = {
    "rmse": tf.contrib.learn.MetricSpec(
        metric_fn=tf.contrib.metrics.streaming_root_mean_squared_error)
}

def init_monitors():
    early_stop_monitor = tf.contrib.learn.monitors.ValidationMonitor(
        X_val,
        y_val,
        every_n_steps=50,
        metrics=validation_metrics,
        early_stopping_metric="rmse",
        early_stopping_metric_minimize=True,
        early_stopping_rounds=500)

    return [early_stop_monitor]

FEATURE_COLUMNS = infer_real_valued_columns_from_input(X_train)
MODEL_DIR_BASE = "./checkpoints/DNNRegressor"

NUM_EPOCHS = 20
BATCH_SIZE = 256
NUM_BATCHES = int(num_train/BATCH_SIZE)

### Create a simple neural network model and make sure we can overfit small data batch

In [13]:
tf.logging.set_verbosity(tf.logging.INFO)
model = tf.contrib.learn.DNNRegressor(
    feature_columns=feature_columns,
    hidden_units=[50, 50],
    label_dimension=D_out,
    activation_fn=tf.nn.relu,
    optimizer=tf.train.AdamOptimizer(
      learning_rate=1e-2,
    )
)

model.fit(x=X_dev[:10], y=y_train[:10], steps=500)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x14763fa58>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': None}
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving int

/Users/kristian/code/berkeley-cs294/homework1/.venv/lib/python3.6/site-packages/tensorflow/python/util/deprecation.py:248: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  equality = a == b


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/_l/_msj97l55wsghyt1qh_r0jyh0000gn/T/tmp2wxahu4m/model.ckpt.
INFO:tensorflow:loss = 0.578984, step = 1
INFO:tensorflow:global_step/sec: 490.482
INFO:tensorflow:loss = 6.76339e-07, step = 101 (0.205 sec)
INFO:tensorflow:global_step/sec: 311.606
INFO:tensorflow:loss = 2.11685e-11, step = 201 (0.321 sec)
INFO:tensorflow:global_step/sec: 498.688
INFO:tensorflow:loss = 9.20041e-15, step = 301 (0.200 sec)
INFO:tensorflow:global_step/sec: 481.602
INFO:tensorflow:loss = 8.15142e-15, step = 401 (0.208 sec)
INFO:tensorflow:Saving checkpoints for 500 into /var/folders/_l/_msj97l55wsghyt1qh_r0jyh0000gn/T/tmp2wxahu4m/model.ckpt.
INFO:tensorflow:Loss for final step: 9.53764e-15.


DNNRegressor(params={'head': <tensorflow.contrib.learn.python.learn.estimators.head._RegressionHead object at 0x14763f5f8>, 'hidden_units': [50, 50], 'feature_columns': (_RealValuedColumn(column_name='', dimension=111, default_value=None, dtype=tf.float32, normalizer=None),), 'optimizer': <tensorflow.python.training.adam.AdamOptimizer object at 0x14763fdd8>, 'activation_fn': <function relu at 0x131e53730>, 'dropout': None, 'gradient_clip_norm': None, 'embedding_lr_multipliers': None, 'input_layer_min_slice_size': None})

### Initial baseline model

In [63]:
def create_baseline_model():
    model = tf.contrib.learn.DNNRegressor(
        model_dir=MODEL_DIR_BASE + "-baseline",
        feature_columns=FEATURE_COLUMNS,
        hidden_units=[20],
        label_dimension=D_out,
        activation_fn=tf.nn.relu,
        dropout=0.0,
        optimizer=tf.train.AdamOptimizer(
          learning_rate=1e-2,
        )
    )

    return model

In [ ]:
baseline_model = create_baseline_model()

eval_history = []

monitors = init_monitors()
for epoch in range(NUM_EPOCHS):
    baseline_model.fit(
        x=X_train,
        y=y_train,
        monitors=monitors,
        steps=num_train
    )
    eval_result = baseline_model.evaluate(x=X_val, y=y_val)
    eval_history.append(eval_history)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x146335240>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': None}
Instructions for updating:
Monitors are deprecated. Please use tf.train.SessionRunHook.
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Ins

/Users/kristian/code/berkeley-cs294/homework1/.venv/lib/python3.6/site-packages/tensorflow/python/util/deprecation.py:248: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  equality = a == b


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into ./checkpoints/DNNRegressor-baseline/model.ckpt.
INFO:tensorflow:loss = 0.61778, step = 1
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on t

INFO:tensorflow:loss = 0.00178105, step = 5801 (5.807 sec)
INFO:tensorflow:global_step/sec: 16.1101
INFO:tensorflow:loss = 0.00176012, step = 5901 (6.207 sec)
INFO:tensorflow:global_step/sec: 15.9287
INFO:tensorflow:loss = 0.00175339, step = 6001 (6.278 sec)
INFO:tensorflow:global_step/sec: 18.1076
INFO:tensorflow:loss = 0.00175456, step = 6101 (5.522 sec)
INFO:tensorflow:global_step/sec: 15.8993
INFO:tensorflow:loss = 0.00179457, step = 6201 (6.290 sec)
INFO:tensorflow:global_step/sec: 16.9293
INFO:tensorflow:loss = 0.00175498, step = 6301 (5.908 sec)
INFO:tensorflow:global_step/sec: 20.7452
INFO:tensorflow:loss = 0.00176674, step = 6401 (4.819 sec)
INFO:tensorflow:global_step/sec: 18.7002
INFO:tensorflow:loss = 0.00178655, step = 6501 (5.348 sec)
INFO:tensorflow:global_step/sec: 19.8586
INFO:tensorflow:loss = 0.00176072, step = 6601 (5.036 sec)
INFO:tensorflow:global_step/sec: 20.5701
INFO:tensorflow:loss = 0.00178449, step = 6701 (4.861 sec)
INFO:tensorflow:global_step/sec: 19.6204


INFO:tensorflow:global_step/sec: 17.3836
INFO:tensorflow:loss = 0.00180587, step = 11601 (5.753 sec)
INFO:tensorflow:global_step/sec: 18.5643
INFO:tensorflow:loss = 0.001757, step = 11701 (5.387 sec)
INFO:tensorflow:global_step/sec: 18.4036
INFO:tensorflow:loss = 0.00176398, step = 11801 (5.434 sec)
INFO:tensorflow:global_step/sec: 19.9161
INFO:tensorflow:loss = 0.00182816, step = 11901 (5.021 sec)
INFO:tensorflow:global_step/sec: 18.9231
INFO:tensorflow:loss = 0.00174885, step = 12001 (5.285 sec)
INFO:tensorflow:global_step/sec: 19.9139
INFO:tensorflow:loss = 0.00181364, step = 12101 (5.022 sec)
INFO:tensorflow:global_step/sec: 20.0523
INFO:tensorflow:loss = 0.00175003, step = 12201 (4.987 sec)
INFO:tensorflow:global_step/sec: 21.2968
INFO:tensorflow:loss = 0.00180491, step = 12301 (4.695 sec)
INFO:tensorflow:global_step/sec: 19.2943
INFO:tensorflow:loss = 0.00186098, step = 12401 (5.183 sec)
INFO:tensorflow:global_step/sec: 15.9517
INFO:tensorflow:loss = 0.0017562, step = 12501 (6.26

...

Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Starting evaluation at 2017-05-27-04:29:44
INFO:tensorflow:Restoring parameters from ./checkpoints/DNNRegressor-baseline/model.ckpt-17051
INFO:tensorflow:Finished evaluation at 2017-05-27-04:29:45
INFO:tensorflow:Saving dict for global step 17051: global_step = 17051, loss = 0.00217724, rmse = 0.046661
INFO:tensorflow:Validation (step 17100): loss =

INFO:tensorflow:Validation (step 17150): loss = 0.00218019, rmse = 0.0466925, global_step = 17101
INFO:tensorflow:Stopping. Best step: 10450 with rmse = 0.046625107526779175.
INFO:tensorflow:Saving checkpoints for 17150 into ./checkpoints/DNNRegressor-baseline/model.ckpt.
INFO:tensorflow:Loss for final step: 0.00194195.
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Please switch to tf.summary.scalar. Note th

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Starting evaluation at 2017-05-27-04:29:59
INFO:tensorflow:Restoring parameters from ./checkpoints/DNNRegressor-baseline/model.ckpt-17200
INFO:tensorflow:Finished evaluation at 2017-05-27-04:29:59
INFO:tensorflow:Saving dict for global step 17200: global_step = 17200, loss = 0.00217127
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Le

INFO:tensorflow:loss = 0.00191086, step = 20201 (6.277 sec)
INFO:tensorflow:global_step/sec: 15.9834
INFO:tensorflow:loss = 0.00178749, step = 20301 (6.256 sec)
INFO:tensorflow:global_step/sec: 15.9976
INFO:tensorflow:loss = 0.00174144, step = 20401 (6.251 sec)
INFO:tensorflow:global_step/sec: 16.0593
INFO:tensorflow:loss = 0.00173046, step = 20501 (6.227 sec)
INFO:tensorflow:global_step/sec: 15.9843
INFO:tensorflow:loss = 0.00172379, step = 20601 (6.256 sec)
INFO:tensorflow:global_step/sec: 16.2601
INFO:tensorflow:loss = 0.00172301, step = 20701 (6.150 sec)
INFO:tensorflow:global_step/sec: 16.0914
INFO:tensorflow:loss = 0.00177416, step = 20801 (6.214 sec)
INFO:tensorflow:global_step/sec: 15.931
INFO:tensorflow:loss = 0.00174242, step = 20901 (6.277 sec)
INFO:tensorflow:global_step/sec: 16.0303
INFO:tensorflow:loss = 0.00172125, step = 21001 (6.238 sec)
INFO:tensorflow:global_step/sec: 15.9412
INFO:tensorflow:loss = 0.00175857, step = 21101 (6.273 sec)
INFO:tensorflow:global_step/sec:

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Starting evaluation at 2017-05-27-04:40:05
INFO:tensorflow:Restoring parameters from ./checkpoints/DNNRegressor-baseline/model.ckpt-27552
INFO:tensorflow:Finished evaluation at 2017-05-27-04:40:05
INFO:tensorflow:Saving dict for global step 27552: global_step = 27552, loss = 0.00205905, rmse = 0.0453767
INFO:tensorflow:Validation (step 27600): loss = 0.00205905, rmse = 0.0453767, global_step = 27552
INFO:tensorflow:global_step/sec: 14.4406
INFO:tensorflow:loss = 0.00170248, step = 27601 (6.924 sec)
INFO:tensorflow:global_step/sec: 16.3349
INFO:tensorflow:loss = 0.00176099, step = 27701 (6.122 sec)
INFO:tensorflow:global_step/sec: 16.2781
INFO:tens

### Multi-layer model

In [25]:
def create_multilayer_model(hidden_units, learning_rate, dropout):
    model_dir = ("{}-hidden_units-{}-learning_rate-{}-dropout-{}"
                "".format(MODEL_DIR_BASE,
                          "_".join(map(str, hidden_units)),
                          learning_rate,
                          dropout))
    
    model = tf.contrib.learn.DNNRegressor(
        model_dir=model_dir,
        feature_columns=FEATURE_COLUMNS,
        hidden_units=hidden_units,
        label_dimension=D_out,
        activation_fn=tf.nn.relu,
        dropout=dropout,
        optimizer=tf.train.AdamOptimizer(
          learning_rate=1e-2,
        )
    )
    
    return model

In [ ]:
multilayer_model = create_multilayer_model([50, 50, 50],
                                           1e-3,
                                           0.0)

eval_history = []

monitors = init_monitors()
for epoch in range(NUM_EPOCHS):
    multilayer_model.fit(
        x=X_train,
        y=y_train,
        monitors=monitors,
        steps=3
    )
    eval_result = multilayer_model.evaluate(x=X_val, y=y_val)
    eval_history.append(eval_history)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x14c544f28>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': None}
Instructions for updating:
Monitors are deprecated. Please use tf.train.SessionRunHook.
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Ins

/Users/kristian/code/berkeley-cs294/homework1/.venv/lib/python3.6/site-packages/tensorflow/python/util/deprecation.py:248: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  equality = a == b


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into ./checkpoints/DNNRegressor-hidden_units-50_50_50-learning_rate-0.001-dropout-0.0/model.ckpt.
INFO:tensorflow:loss = 0.194544, step = 1
INFO:tensorflow:Saving checkpoints for 3 into ./checkpoints/DNNRegressor-hidden_units-50_50_50-learning_rate-0.001-dropout-0.0/model.ckpt.
INFO:tensorflow:Loss for final step: 0.0861178.
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCom

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from ./checkpoints/DNNRegressor-hidden_units-50_50_50-learning_rate-0.001-dropout-0.0/model.ckpt-6
INFO:tensorflow:Saving checkpoints for 7 into ./checkpoints/DNNRegressor-hidden_units-50_50_50-learning_rate-0.001-dropout-0.0/model.ckpt.
INFO:tensorflow:loss = 0.037718, step = 7
INFO:tensorflow:Saving checkpoints for 9 into ./checkpoints/DNNRegressor-hidden_units-50_50_50-learning_rate-0.001-dropout-0.0/model.ckpt.
INFO:tensorflow:Loss for final step: 0.0318.
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from ./checkpoints/DNNRegressor-hidden_units-50_50_50-learning_rate-0.001-dropout-0.0/model.ckpt-12
INFO:tensorflow:Saving checkpoints for 13 into ./checkpoints/DNNRegressor-hidden_units-50_50_50-learning_rate-0.001-dropout-0.0/model.ckpt.


### Train simple neural network model with the training data